In [26]:
import os
import json
import pandas as pd

In [27]:
def load_json_file(file):
    with open(file) as f:
        data = json.load(f)
    return data['data']

In [28]:
directory = "cro"
file_list = os.listdir(directory)

In [29]:
df = pd.DataFrame()
frames = []
for file in file_list:
    try:
        if file.endswith(".json"):
            filepath = os.path.join(directory, file)
            stanice = file.split("-")[0]
            dataframe = pd.DataFrame(load_json_file(filepath))
            dataframe['stanice'] = stanice
            frames.append(dataframe)
    except:
        print(file)

In [30]:
df = pd.concat(frames)

In [31]:
df = df.drop(columns=['files','itemcode']).sort_values(by='since').reset_index(drop=True)

In [32]:
df

,since,id,interpret,interpret_id,track,track_id,stanice
0,2023-10-22T00:00:22+02:00,19376269.0,SYLVAN ESSO,11101.0,Frequency,82943.0,radiowave
1,2023-10-22T00:03:12+02:00,19376274.0,WINONA OAK,25899.0,Winter Rain,87407.0,radiowave
2,2023-10-22T00:07:28+02:00,19376280.0,DIIV,2909.0,Skin Game,69599.0,radiowave
3,2023-10-22T00:10:40+02:00,19376285.0,ROBBIE WILLIAMS,2984.0,ROAD TO MANDALAY,7475.0,radiozurnal
4,2023-10-22T00:11:24+02:00,19376286.0,DUA LIPA,11139.0,DANCE THE NIGHT,108512.0,dvojka
...,...,...,...,...,...,...,...
219417,2024-08-27T23:53:31+02:00,20843091.0,JESSIE J,9523.0,I Want Love,91200.0,radiowave
219418,2024-08-27T23:53:35+02:00,20843092.0,OASIS,6426.0,WHATEVER,56027.0,dvojka
219419,2024-08-27T23:56:46+02:00,20843099.0,U.S. GIRLS,9345.0,So Typically Now,102167.0,radiowave
219420,2024-08-27T23:57:04+02:00,20843101.0,PICTURE THIS,31599.0,GET ON MY LOVE,105566.0,radiozurnal


In [33]:
df.to_json(os.path.join("data","cro_raw.json"), orient="records")